In [ ]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import shutil
from shutil import copyfile
import matplotlib.pyplot as plt
from PIL import Image
import ultralytics
from ultralytics import YOLO

In [ ]:
ultralytics.checks()

In [ ]:
root_path = "./podaci_detekcija/"
default_path = './podaci_detekcija/ts/ts'
train_path = './podaci_detekcija/train/'
valid_path = './podaci_detekcija/valid/'
test_path = './podaci_detekcija/test/'

In [ ]:
os.makedirs(os.path.join(root_path, "images", "train"), exist_ok=True)
os.makedirs(os.path.join(root_path, "images", "valid"), exist_ok=True)
os.makedirs(os.path.join(root_path, "images", "test"), exist_ok=True)

In [ ]:
os.makedirs(os.path.join(root_path, "labels", "train"), exist_ok=True)
os.makedirs(os.path.join(root_path, "labels", "valid"), exist_ok=True)
os.makedirs(os.path.join(root_path, "labels", "test"), exist_ok=True)

In [ ]:
image_files = [f for f in os.listdir(default_path) if f.endswith('.jpg')]

In [ ]:
random.shuffle(image_files)

In [ ]:
train_ratio = 0.7
valid_ratio = 0.2
test_ratio = 0.1
total_samples = len(image_files)
train_split = int(train_ratio * total_samples)
valid_split = int((1-(train_ratio + test_ratio)) * total_samples)
train_files = image_files[:train_split]
valid_files = image_files[train_split:train_split+valid_split]
test_files = image_files[train_split+valid_split:]

In [ ]:
print(f"Ukupno slika: {total_samples}")
print(f"Trening slike: {len(train_files)}")
print(f"Validacijske slike: {len(valid_files)}")
print(f"Test slike: {len(test_files)}")
print("---------------------------------------------")
print(train_files)
print(valid_files)
print(test_files)

In [ ]:
for file in train_files:
  copyfile(os.path.join(default_path, file), os.path.join(root_path, "images", "train", file))
  copyfile(os.path.join(default_path, file.replace('.jpg', '.txt')), os.path.join(root_path, "labels", "train", file.replace('.jpg', '.txt')))
    
for file in valid_files:
  copyfile(os.path.join(default_path, file), os.path.join(root_path, "images", "valid", file))
  copyfile(os.path.join(default_path, file.replace('.jpg', '.txt')), os.path.join(root_path, "labels", "valid", file.replace('.jpg', '.txt')))
    
for file in test_files:
  copyfile(os.path.join(default_path, file), os.path.join(root_path, "images", "test", file))
  copyfile(os.path.join(default_path, file.replace('.jpg', '.txt')), os.path.join(root_path, "labels", "test", file.replace('.jpg', '.txt')))

In [ ]:
import yaml

data_yaml = dict(
    train = 'train ',
    val = 'valid  ',
    test = 'test',
    nc = 4,
    names =['prohibitor','danger','mandatory','other']
)

with open('data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

In [ ]:
names = ['prohibitor','danger','mandatory','other']
M = list(range(len(names)))
class_map = dict(zip(M,names))

In [ ]:
model = YOLO("yolov8x.pt") 

In [ ]:
results = model.train(data='./data.yaml', epochs=10, imgsz=480)
# C:\Users\DT User3\Desktop\AjdinBukvic_POOS_Projekat\podaci_detekcija\images